# Burry

Burry is a project to analyse text data, in the form of tweets, to find relation between those texts and subjects.

It is done as a project for the Fontys University of ICT and the Central Bureau of Statistics of the Netherlands.

Currently it is focussed at detecting sickness.

In [3]:
import pandas as pd
import numpy as np
import operator
import re
from collections import Counter
from difflib import SequenceMatcher
from urllib.parse import urlencode
import webbrowser
import gensim
import bz2
import logging
from gensim import corpora, models, similarities
from gensim.corpora import WikiCorpus
from gensim.models.ldamodel import LdaModel
from gensim.models.word2vec import LineSentence, Word2Vec

doc = pd.read_csv('sample_dataset.csv', sep=';')

search_terms = [re.sub(r'(src:\w+)', '', word).strip() for word in doc['zoekopdracht'].unique()]
print(search_terms)

sickness_terms = [
    'ziek', 
    'griep', 
    'verkouden', 
    'verkoudheid',
    'koorts', 
    'hoofdpijn',
]

doc.head()

['ziek']


,zoekopdracht,datum,url,sentiment,type,discussielengte,views,auteur,volgers,invloed,GPS breedtegraad,GPS lengtegraad,bericht tekst,type bron,titel
0,ziek src:twitter,2017-09-14 10:26,https://twitter.com/dekoran1/status/9082454469...,NaN,post,NaN,11369.0,dekoran1,11369,131.1,NaN,NaN,@BLIKOPENER333,twitter,NaN
1,ziek src:twitter,2017-09-14 10:25,https://twitter.com/BLIKOPENER333/status/90824...,-,comment,2.0,17592.0,BLIKOPENER333,17592,9.1,NaN,NaN,RT @tikkierechts: Ziek of niet dit is echt een...,twitter,NaN
2,ziek src:twitter,2017-09-14 10:24,https://twitter.com/Jesske3003/status/90824496...,-,comment,19.0,NaN,Jesske3003,856,4.5,NaN,NaN,@tinamieneke @DockxGunter Ja en elkaar wat zie...,twitter,NaN
3,ziek src:twitter,2017-09-14 10:23,https://twitter.com/tinamieneke/status/9082446...,-,comment,19.0,NaN,tinamieneke,1928,25.7,NaN,NaN,"@DockxGunter @Jesske3003 Ik vind dat niet oké,...",twitter,NaN
4,ziek src:twitter,2017-09-14 10:23,https://twitter.com/BeQuanFRIvoedin/status/908...,-,post,NaN,38.0,BeQuanFRIvoedin,38,0.6,NaN,NaN,Hippe huiskip maakte nog nooit zoveel Amerikan...,twitter,NaN


# Cleaning data
Here we do several things:
* remove retweets
* convert text to lowercase

In the context of our research a retweet does not add any relevant data. It actually skews the results since a specific phrasing get's repeated more often than it naturally would.

In [5]:
doc['bericht tekst'] = doc['bericht tekst'].fillna('')

In [6]:
doc['bericht tekst'] = doc['bericht tekst'].str.lower()

In [28]:
doc  = doc[~doc['bericht tekst'].str.contains('rt')]
doc  = doc[~doc['bericht tekst'].str.contains('http')]  # TODO secure?
doc  = doc[~doc['auteur'].str.contains('grieptweets')]  # TODO to file
doc  = doc[~doc['auteur'].str.contains('kleenex_helpt')]
# doc.head()

In [8]:
doc = doc.drop_duplicates()
len(doc)

3591

# Defenition of helper function


In [12]:
re_clean = re.compile(r'(https?://\S+|@\S+)')
re_words = re.compile(r'(\w+-?\w*)')

def clean_text(text: str):
    words = []
    if text:
        text = re_clean.sub(' ', text)
        words = re_words.findall(text)
    return words

In [13]:
all_tweets = pd.read_csv('sample_alltweets.csv', sep=';')
all_tweets.head()

,zoekopdracht,datum,url,sentiment,type,discussielengte,views,auteur,volgers,invloed,GPS breedtegraad,GPS lengtegraad,bericht tekst,type bron,titel
0,src:twitter,2017-09-14 12:41,https://twitter.com/PotatoEurope/status/908279...,+,comment,2.0,2062.0,PotatoEurope,2062,6.4,NaN,NaN,RT @m_schaftenaar: Het is wat modderig maar de...,twitter,NaN
1,src:twitter,2017-09-14 12:41,https://twitter.com/EditieNL/status/9082795646...,NaN,post,NaN,52117.0,EditieNL,52117,62.7,NaN,NaN,#peticure: nagels lakken van je hond of kat po...,twitter,NaN
2,src:twitter,2017-09-14 12:41,https://twitter.com/SocialeBusiness/status/908...,NaN,post,NaN,1397.0,SocialeBusiness,1397,0.6,NaN,NaN,ThoughtWorks Launches Ventures Group - MarketW...,twitter,NaN
3,src:twitter,2017-09-14 12:41,https://twitter.com/ahnxhaniy/status/908279540...,NaN,comment,2.0,NaN,ahnxhaniy,575,0.4,NaN,NaN,@youcnna Welkam unni,twitter,NaN
4,src:twitter,2017-09-14 12:41,https://twitter.com/_Leeuwarden/status/9082795...,NaN,post,NaN,663.0,_Leeuwarden,663,0.7,NaN,NaN,Fietsendief neemt de benen https://t.co/q2iJ8n...,twitter,NaN


# Cleaning data
Here we do several things:
* convert messages with no content with to the message `''`
* remove retweets
* convert to lowercase

The conversion is needed since the `clean_text()` function expects a string.

In [14]:
all_tweets['bericht tekst'] = all_tweets['bericht tekst'].fillna('')

In [15]:
all_tweets['bericht tekst'] = all_tweets['bericht tekst'].str.lower()

In [16]:
all_tweets = all_tweets[~all_tweets['bericht tekst'].str.contains('rt')]
all_tweets.head()

,zoekopdracht,datum,url,sentiment,type,discussielengte,views,auteur,volgers,invloed,GPS breedtegraad,GPS lengtegraad,bericht tekst,type bron,titel
1,src:twitter,2017-09-14 12:41,https://twitter.com/EditieNL/status/9082795646...,NaN,post,NaN,52117.0,EditieNL,52117,62.7,NaN,NaN,#peticure: nagels lakken van je hond of kat po...,twitter,NaN
2,src:twitter,2017-09-14 12:41,https://twitter.com/SocialeBusiness/status/908...,NaN,post,NaN,1397.0,SocialeBusiness,1397,0.6,NaN,NaN,thoughtworks launches ventures group - marketw...,twitter,NaN
3,src:twitter,2017-09-14 12:41,https://twitter.com/ahnxhaniy/status/908279540...,NaN,comment,2.0,NaN,ahnxhaniy,575,0.4,NaN,NaN,@youcnna welkam unni,twitter,NaN
5,src:twitter,2017-09-14 12:41,https://twitter.com/gerritwermink/status/90827...,+,comment,3.0,NaN,gerritwermink,2205,13.9,NaN,NaN,@dukkie6 shitverr... zeker een vaarskalf... va...,twitter,NaN
6,src:twitter,2017-09-14 12:41,https://twitter.com/ToysAndGamesCo/status/9082...,NaN,post,NaN,3800.0,ToysAndGamesCo,3800,0.6,NaN,NaN,"great price: $346.55, geeetech i3 pro c 3d pr...",twitter,NaN


In [17]:
all_tweets['bericht woorden'] = all_tweets['bericht tekst'].map(clean_text)

# Learning

Now we start learning from our dataset.

First we will find the most common words in our set, so we know which ones to ignore. Think like "the" and "it" in english.

We will put these word's in our blacklist anlong with our search terms.

We are using the dataset without filters to scan all possible tweets.


In [19]:
counter = Counter()

for words in all_tweets['bericht woorden']:
    counter.update(words)
counter.most_common(100)

[('de', 1021),
 ('in', 766),
 ('is', 644),
 ('een', 628),
 ('en', 613),
 ('van', 600),
 ('het', 502),
 ('ik', 483),
 ('op', 437),
 ('voor', 389),
 ('je', 384),
 ('met', 367),
 ('dat', 340),
 ('the', 334),
 ('niet', 290),
 ('te', 273),
 ('of', 226),
 ('to', 214),
 ('a', 209),
 ('zijn', 208),
 ('die', 208),
 ('ook', 203),
 ('i', 198),
 ('s', 196),
 ('maar', 196),
 ('er', 193),
 ('via', 190),
 ('aan', 184),
 ('wat', 178),
 ('bij', 161),
 ('om', 157),
 ('you', 155),
 ('over', 144),
 ('dan', 143),
 ('and', 135),
 ('for', 134),
 ('we', 131),
 ('nog', 131),
 ('wel', 131),
 ('als', 130),
 ('2', 125),
 ('nu', 125),
 ('naar', 124),
 ('it', 123),
 ('zo', 121),
 ('on', 115),
 ('ze', 111),
 ('haar', 110),
 ('mclisse', 108),
 ('was', 107),
 ('dit', 103),
 ('1', 102),
 ('mijn', 101),
 ('ontgsblogcon', 101),
 ('weer', 100),
 ('uit', 99),
 ('door', 97),
 ('kan', 95),
 ('geen', 95),
 ('heb', 94),
 ('meer', 93),
 ('ja', 89),
 ('12', 86),
 ('gaat', 85),
 ('al', 85),
 ('me', 84),
 ('heeft', 82),
 ('at', 81

In [20]:
common_words = set([word[0] for word in counter.most_common(300)])

In [21]:
blacklisted_words = set(common_words)
blacklisted_words.update(set(search_terms))

# Related words
Now we will try to find related words to our search terms.

We will be doing this using our blacklist to filter out the blacklisted and common words.

Essentially this also is a common_words list but then to find the relations instead of common words.

In [22]:
doc['bericht woorden'] = doc['bericht tekst'].map(clean_text)
counter = Counter()

for words in doc['bericht woorden']:
    words = set(words)
    filtered_words = words - blacklisted_words
    counter.update(filtered_words)
    
related_words = counter.most_common(25)
related_words

[('bent', 160),
 ('word', 132),
 ('thuis', 111),
 ('chronisch', 106),
 ('eigen', 103),
 ('risico', 77),
 ('erg', 76),
 ('omdat', 75),
 ('eigenrisico', 72),
 ('voel', 67),
 ('morgen', 62),
 ('bed', 59),
 ('beetje', 55),
 ('beter', 55),
 ('helemaal', 54),
 ('werken', 50),
 ('iedereen', 49),
 ('dagen', 46),
 ('geweest', 46),
 ('betalen', 45),
 ('kind', 44),
 ('geworden', 44),
 ('beterschap', 44),
 ('hele', 44),
 ('hebt', 43)]

# Training
Now we will train the model using a dataset from wikipedia.

This dataset is used to replicate normal language and analyse the relations between words.

We have already trained a model and will be loading it in here

In [23]:
model = Word2Vec.load('word2vec.model')

# Analysing
Now we will analyse our dataset using our model.

We will check for similarity of the tweets against our sickness_terms to score a tweet to being from a sick person.


In [24]:
def scorer(row, words):
    if 'score' not in row:
        score = 0
        words = [word.replace('#', '') for word in row['bericht woorden'] if word in model.wv.vocab]
        if words:
            score = model.wv.n_similarity(sickness_terms, words)
        if row['type'] == 'comment':
            score /= 2
        row['score'] = score
    return row

# words = [word[0] for word in related_words]
doc = doc.apply(scorer, axis=1, words=words)

In [25]:
doc = doc.sort_values('score', ascending=False)

In [26]:
pd.set_option('display.max_colwidth', 250)
pd.options.display.max_rows = 999

# Visualization
As you can see below, the model is pretty good at finding the relevant tweets.

The problem we face now is that Dutch people like the swear with deceases.
 - Ziek volkje -> sick people (in the sense of not thinking straight)
 - Tering ziek -> Tuberculosis sick (ambiguous: could be really sick, or I can't understand why)

In [27]:
doc.filter(items=['bericht tekst', 'auteur', 'type', 'score']).head(250)

,bericht tekst,auteur,type,score
2500,voor 2e dag ziek thuis. maag/darmen teveel prikkels. heerst er weer iets? #buikgriep #griep #ziek #ziekthuis #zzm #ziekzwakmisselijk #gatver,johnkapjr,post,0.787678
3391,"hoofdpijn, buikpijn, moe tekenen van ziek komen??😞",FireflyChar90,post,0.761952
2865,ziek veel stress,cedric_vandun,post,0.759097
7373,ziek volkje.,ronald_brok,post,0.744404
7507,tering ziek..,sel0reos,post,0.741515
7929,smerig ziek klotevolk,swimmingsigs,post,0.738240
7962,ziek thuis met griep,SiennaLetterie,post,0.732541
6079,#ziek doorgedraaid lijstjessysteem.,atempestteacup,post,0.728860
1477,"kapot misselijk, ziek erge hoofdpijn & kan niet slapen. nice",dxsmg_,post,0.728676
2957,ziek.,Elsvtol,post,0.727964
